# Library

In [22]:
import requests
from bs4 import BeautifulSoup as bs

import time, os, random
import pandas as pd
import numpy as np

from selenium import webdriver
import chromedriver_autoinstaller as ca

from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## chrome driver 설치

In [23]:
# USB error 메세지 발생 해결을 위한 코드
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])

In [24]:
# 현재 크롬 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'131'

In [25]:
# # 크롬 드라이버 확인 및 설치(처음 한번만 실행)
# ca.install(True)

## chrome driver 실행

In [26]:
# 웹드라이버 실행
driver = webdriver.Chrome(options=options)

# 페이지 접속

In [28]:
# 페이지 주소
url = 'https://news.naver.com/section/100' # 네이버 뉴스 > 정치

## selenium 작동

In [29]:
driver.get(url)
driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
# driver.maximize_window() # 브라우져 창 최대화

## 헤드라인 뉴스 섹션 로드

In [30]:
# 헤드라인 뉴스
headline_news_more_view_button_xpath = '//*[@id="newsct"]/div[1]/div[2]/a' # 헤드라인 더 보기 버튼

# 헤드라인 더 보기 버튼 로딩 대기
headline_news_more_view_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, headline_news_more_view_button_xpath))
)

# 헤드라인 더보기 버튼 클릭
headline_news_more_view_button.click()
driver.implicitly_wait(10)

# 헤드라인 뉴스 수집

In [31]:
page = driver.page_source
soup = bs(page, 'html.parser')
soup.title.text

'정치 : 네이버 뉴스'

In [32]:
# 헤드라인 뉴스 섹션
headline_news_section = soup.select_one('ul.sa_list')
len(headline_news_section)

21

In [33]:
# 뉴스 타이들 리스트
headline_news_list = headline_news_section.select('li.sa_item._SECTION_HEADLINE')
len(headline_news_list)

10

# 수집한 데이터 전처리

In [34]:
link_list = []
title_list = []
content_list = []
press_list = []
related_news_count_list = []

for i in range(len(headline_news_list)):
    # 기사 내용 섹션
    news_content_section = headline_news_list[i].select_one('div.sa_text')

    # 기사 링크
    try:
        link = news_content_section.select_one('a.sa_text_title._NLOG_IMPRESSION')['href']
    except:
        link = np.nan
        print(i)
        # print('기사 링크 없음')
    
    # 타이틀
    try:
        title = news_content_section.select_one('strong.sa_text_strong').text
    except:
        title = np.nan
        print(i)
        # print('타이틀 없음')
    
    # 기사 내용
    try:
        content = news_content_section.select_one('div.sa_text_lede').text
    except:
        content = np.nan
        print(i)
        # print('기사 내용 없음')
    
    ## 언론사
    try:
        press = news_content_section.select_one('div.sa_text_press').text
    except:
        press = np.nan
        print(i)
        # print('언론사 없음')
    
    ## 관련 뉴스 개수
    try:
        related_news_count = int(news_content_section.select_one('span.sa_text_cluster_num').text)
    except:
        related_news_count = np.nan
        print(i)
        # print('관련 뉴스 개수 없음')
    
    # 정보 리스트에 담기
    link_list.append(link)
    title_list.append(title)
    content_list.append(content)
    press_list.append(press)
    related_news_count_list.append(related_news_count)

In [35]:
len(link_list), len(title_list), len(content_list), len(press_list), len(related_news_count_list)

(10, 10, 10, 10, 10)

In [36]:
data_dict = {
    '번호': range(1, len(link_list)+1),
    '기사 링크': link_list,
    '기사 제목': title_list,
    '기사 내용': content_list,
    '언론사': press_list,
    '관련 뉴스 개수': related_news_count_list
    }
df = pd.DataFrame(data_dict)
df.head()

,번호,기사 링크,기사 제목,기사 내용,언론사,관련 뉴스 개수
0,1,https://n.news.naver.com/mnews/article/025/000...,"""北도발 징후 입체적 식별 가능""…軍정찰위성 3호기 발사",한국군 독자 정찰위성 3호기 발사가 21일 성공적으로 실시됐다. 7년 전 시작된 이...,중앙일보,83
1,2,https://n.news.naver.com/mnews/article/003/001...,"장갑차 맨몸으로 막은 남성…이재명 ""이 분 꼭 찾아달라""",이재명 더불어민주당 대표가 지난 3일 비상계엄 선포 당시 국회로 출동한 군용 차량을...,뉴시스,32
2,3,https://n.news.naver.com/mnews/article/449/000...,"권성동 “내란·김여사 특검법, 野 국정·여당 마비 속셈”",권성동 국민의힘 대표 권한대행 겸 원내대표는 오늘(22일) 내란 특검법과 김건희 여...,채널A,134
3,4,https://n.news.naver.com/mnews/article/088/000...,"조국당, 한덕수 탄핵소추안 공개…""내란 세력의 준동 제압해야""",조국혁신당이 22일 한덕수 대통령 권한대행 겸 국무총리에 대한 탄핵소추안을 공개했다...,매일신문,36
4,5,https://n.news.naver.com/mnews/article/015/000...,"'이재명은 안 된다' 현수막 금지…나경원 ""이러니 부정선거 의심""",중앙선거관리위원회가 '내란 공범' 문구가 포함된 야당 현수막은 허용하고 '이재명은 ...,한국경제,106


# 데이터 출력

In [37]:
current_date = time.strftime('%Y%m%d')
current_date

'20241223'

In [38]:
current_datetime = time.strftime('%Y%m%d_%p_%I%M%S')
current_datetime

'20241223_AM_121109'

In [39]:
# 현재 경로 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver'

In [40]:
# 수집한 파일 저장할 폴더 생성
crawled_folder_path = os.path.join(code_path, 'crawled_data', 'naver_news', 'politics', current_date)
crawled_folder_path

'c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver\\crawled_data\\naver_news\\politics\\20241223'

In [41]:
os.makedirs(crawled_folder_path, exist_ok=True)

In [42]:
# 엑셀 파일로 출력
df.to_excel(os.path.join(crawled_folder_path, f'naver_news_politics_{current_datetime}.xlsx'), index=False)

In [ ]:
driver.quit()
print('수집 완료')

# END